In [ ]:
%matplotlib inline

proj_path = "/Users/andrew/Desktop/projects/hidden_singles_public/"

In [ ]:
import sys
sys.path.append(proj_path + 'python')

import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import json

from hiddensingles.misc import utils
from hiddensingles.misc.draw_sudoku import *
from hiddensingles.experiment.data_wrangler import load_raw_data
from hiddensingles.sudoku.grid import Grid, GridString, Coordinate

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 300)

In [ ]:
def get_coordinates(hidden_single_raw):
    coords = {}
    for k, v in hidden_single_raw['coordinates'].items():
        if type(v) == list:
            coords[k] = [Coordinate(c['x'], c['y']) for c in v]
        else:
            coords[k] = Coordinate(v['x'], v['y'])
    return coords

def render_hidden_single(hidden_single_raw, goal_entry=None, size=400):
    house_type_key = 'house_type' if 'house_type' in hidden_single_raw else 'houseType'
    house_type = hidden_single_raw[house_type_key]
    gridstring = hidden_single_raw['gridstrings']['puzzle']
    coordinates = get_coordinates(hidden_single_raw)
    goal = coordinates['goal']
    cell_colors = get_house_highlights(house_type, goal)

    # Format highlights as dictionary: {(x, y): (R, G, B)}
    column = house_type == "Column"
    cell_colors[goal] = c_bg_green

    digit_colors = {}
    if goal_entry is not None:
        gridstring = GridString.load(hidden_single_raw['gridstrings']['puzzle'])
        gridstring.set(coordinates['goal'], goal_entry)
        gridstring = str(gridstring)
        digit_colors[coordinates['goal']] = (0, 0, 0)
    return render_sudoku(gridstring, cell_colors, digit_colors=digit_colors, size=size)

# Load Data

In [ ]:
# test data as generated for the screenshots

with open(proj_path + 'data/experiment_new_test.json') as f:
    test_data = json.load(f)

In [ ]:
# actual participants' data
# Load raw_data and map them using sid_hash

raw_data = load_raw_data(proj_path + 'data/raw')
sid_hashes = utils.short_hash([d['worker_id'] for d in raw_data.values()], 6)
raw_data = {sid_hashes[k]: v for k, v in raw_data.items()}

subject_data = pd.read_csv(proj_path + 'data/processed/subject_data.tsv', sep='\t')
experiment = raw_data[sid_hashes[1]]['experimentDetails'] # all figures generated using this person's data

# Exercises

In [ ]:
tutorial = test_data['tutorial']
tut_coords = get_coordinates(tutorial)
goal_coords = tut_coords['goal']

contradiction = test_data['contradiction']
img = render_sudoku(contradiction)
img.save(proj_path + 'figures/sample_puzzles/contradiction.png')

In [ ]:
fullhouse = test_data['fullhouse']
highlights = {goal_coords: c_bg_green}
img = render_sudoku(fullhouse, highlights)
img.save(proj_path + 'figures/sample_puzzles/fullhouse.png')

# Tutorial

In [ ]:
# The actual tutorial puzzle
gridstring = tutorial['gridstrings']['puzzle']
highlights = get_house_highlights(tutorial['house_type'], goal_coords)
highlights[goal_coords] = c_bg_green
img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_0.png')
img = img_add_title(img, 'Tutorial')
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_0_titled.png')

In [ ]:
# Tutorial screen 1 & 5

grid = GridString.load(tutorial['gridstrings']['puzzle']).to_grid()
dist1 = tut_coords['distractor_single']
dist2 = tut_coords['distractor_double']
dist3 = tut_coords['distractor_box']
for coord in [dist1, dist2, dist3]:
    grid.remove(coord)
gridstring = str(grid.to_gridstring())

highlights = get_house_highlights(tutorial['house_type'], goal_coords)
highlights[goal_coords] = c_bg_green

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_1.png')

In [ ]:
# Tutorial screen 2
highlights[tut_coords['empty_double']] = c_bg_purple
for coord in tut_coords['occupied']:
    highlights[coord] = c_bg_red

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_2.png')

In [ ]:
# Tutorial screen 3
highlights[tut_coords['empty_double']] = c_bg_red
for coord in tut_coords['empty_box']:
    highlights[coord] = c_bg_purple

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_3.png')

In [ ]:
# Tutorial screen 4
highlights[tut_coords['empty_single']] = c_bg_purple
for coord in tut_coords['empty_box']:
    highlights[coord] = c_bg_red

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_4.png')

In [ ]:
# Tutorial screen 6
gridstring = tutorial['gridstrings']['puzzle']
highlights = get_house_highlights(tutorial['house_type'], goal_coords)
highlights[goal_coords] = c_bg_green
for coord in tut_coords['occupied']:
    highlights[coord] = c_bg_red
highlights[tut_coords['distractor_double']] = c_bg_purple

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_6.png')

In [ ]:
# Tutorial screen 7
highlights[tut_coords['distractor_double']] = None
highlights[tut_coords['empty_double']] = c_bg_red
highlights[tut_coords['distractor_box']] = c_bg_purple

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_7.png')

In [ ]:
# Tutorial screen 8
highlights[tut_coords['distractor_box']] = None
for coord in tut_coords['empty_box']:
    highlights[coord] = c_bg_red

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_8.png')

In [ ]:
# Tutorial screen 9a
grid = GridString.load(tutorial['gridstrings']['puzzle']).to_grid()
highlights = get_house_highlights(tutorial['house_type'], goal_coords)
highlights[goal_coords] = c_bg_green
highlights[tut_coords['target_single']] = c_bg_orange
highlights[tut_coords['target_double']] = c_bg_orange
highlights[tut_coords['empty_double']] = c_bg_orange
highlights[tut_coords['empty_single']] = c_bg_orange
highlights[tut_coords['target_box']] = c_bg_purple
for coord in tut_coords['empty_box']:
    highlights[coord] = c_bg_purple
for coord in tut_coords['occupied']:
    highlights[coord] = c_bg_red

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_9a.png')

In [ ]:
# Tutorial screen 9b
grid = GridString.load(tutorial['gridstrings']['puzzle']).to_grid()
highlights = get_house_highlights(tutorial['house_type'], goal_coords)
highlights[goal_coords] = c_bg_green
# highlights[tut_coords['distractor_single']] = c_bg_orange
highlights[tut_coords['distractor_double']] = c_bg_orange
highlights[tut_coords['empty_double']] = c_bg_orange
highlights[tut_coords['distractor_box']] = c_bg_purple
for coord in tut_coords['empty_box']:
    highlights[coord] = c_bg_purple
for coord in tut_coords['occupied']:
    highlights[coord] = c_bg_red

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/hs_tutorial_9b.png')

# Practice Phase

In [ ]:
# Sample puzzles

for i in range(4):
    img = render_hidden_single(test_data['phase1'][i])
    img.save(proj_path + 'figures/sample_puzzles/p1_puzzle_{}.png'.format(i+1))

In [ ]:
# In-House feedback

hidden_single = test_data['phase1'][0]
gridstring = hidden_single['gridstrings']['puzzle']
coords = get_coordinates(hidden_single)
goal = coords['goal']
highlights = get_house_highlights(hidden_single['house_type'], goal)
highlights[goal] = c_bg_green
highlights[coords['occupied'][0]] = c_bg_red
img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/p1_feedback_inhouse.png')

In [ ]:
# Absent Feedback

highlights = get_house_highlights(hidden_single['house_type'], goal)
highlights[goal] = c_bg_green
for coord in coords['occupied']:
    highlights[coord] = c_bg_red
img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/p1_feedback_absent.png')

In [ ]:
# Distractor Feedback

highlights = get_house_highlights(hidden_single['house_type'], goal)
highlights = {k: c_bg_red for k in highlights}
highlights[goal] = c_bg_green
highlights[coords['empty_single']] = c_bg_blue
img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/p1_feedback_distractor.png')

In [ ]:
# Target feedback

highlights = get_house_highlights(hidden_single['house_type'], goal)
highlights[goal] = c_bg_green
for coord in coords['occupied']:
    highlights[coord] = c_bg_red
for coord in coords['empty_box']:
    highlights[coord] = c_bg_purple
highlights[coords['empty_single']] = c_bg_orange
highlights[coords['empty_double']] = c_bg_orange

highlights[coords['target_box']] = c_bg_purple
highlights[coords['target_single']] = c_bg_orange
highlights[coords['target_double']] = c_bg_orange

img = render_sudoku(gridstring, highlights)
img.save(proj_path + 'figures/sample_puzzles/p1_feedback_target.png')

# Test Phase

In [ ]:
# Test Phase

for i in range(64):
    hidden_single = test_data['phase2'][i]
    condition = hidden_single['condition']
    
    title = []
    if condition['house_type']:
        title.append('HT')
    if condition['house_index']:
        title.append('HI')
    if condition['cell_index']:
        title.append('CI')
    if condition['digit_set']:
        title.append('DS')
    
    file_suffix = ''.join(title) if title else 'ctrl'
    title = ' | '.join(title) if title else 'Control'
    
    img = render_hidden_single(hidden_single['hidden_single'])
#     img = img_add_title(img, title)
    img.save(proj_path + 'figures/sample_puzzles/test_puzzles/{}_{}.png'.format(i+1, file_suffix))

# Questionnaire

In [ ]:
utils.mkdir(proj_path + 'figures/questionnaire_puzzles/')
for record in tqdm(subject_data[~subject_data.excluded].to_records()):
    subject_id = record.subject_id
    sid_hash = record.sid_hash
    goal_entry = record.q_puzzle_digit
    hidden_single = raw_data[sid_hash]['experimentDetails']['questionnaire']
    img = render_hidden_single(hidden_single, goal_entry=goal_entry)
    title = 'Questionnaire (Subject {})'.format(subject_id)
    img = img_add_title(img, title)
    img.save(proj_path + 'figures/questionnaire_puzzles/{}.png'.format(subject_id))

# Questionnaire Rater Figures

In [ ]:
# Example puzzle for instructions

grid = Grid(3, 3)
grid.write(Coordinate(4, 0), 2)
grid.write(Coordinate(4, 1), 4)
grid.write(Coordinate(4, 5), 8)
grid.write(Coordinate(7, 4), 3)
grid.write(Coordinate(5, 7), 3)
grid.write(Coordinate(8, 2), 3)
grid.write(Coordinate(1, 1), 6)
grid.write(Coordinate(2, 4), 6)
grid.write(Coordinate(5, 8), 6)
gridstring = str(grid.to_gridstring())

hidden_single = {
    'houseType': 'row',
    'gridstrings': {'puzzle': gridstring},
    'coordinates': {'goal': {'x': 4, 'y': 3}}
}
img = render_hidden_single(hidden_single, goal_entry=3)
img.save(proj_path + 'figures/q_rater/sample_puzzle.png')

In [ ]:
def render_questionnaire(qrater_filename, pdf_filename):
    df = pd.read_csv(qrater_filename, sep='\t')

    imgs = []
    for record in tqdm(df.to_records()):
        subject_id = record.subject_id
        sid_hash = record.sid_hash
        goal_entry = record.q_puzzle_digit
        hidden_single = raw_data[sid_hash]['experimentDetails']['questionnaire']
        img = render_hidden_single(hidden_single, goal_entry=goal_entry)
        title = 'Questionnaire (Subject {})'.format(subject_id)
        img = img_add_title(img, title)
        imgs.append(img)

    imgs[0].save(pdf_filename, save_all = True, quality=100, append_images = imgs[1:])

In [ ]:
render_questionnaire(proj_path + 'data/processed/qrater.tsv',
                     proj_path + 'figures/q_rater/puzzles.pdf')
render_questionnaire(proj_path + 'data/processed/qrater_pilot.tsv',
                     proj_path + 'figures/q_rater/pilot_puzzles.pdf')